# Probabilistic Machine Learning - Project Report
**Course:** Probabilistic Machine Learning (SoSe 2025)  
**Lecturer:** Alvaro Diaz-Ruelas  
**Student(s) Name(s):**  Timm Nicklaus  
**GitHub Username(s):**  t1mmb0  
**Date:**  15.08.25  
**PROJECT-ID:** 13-3NTXXXX  

---


# 1. Introduction



### Motivation
Social indicators are important metrics collected by the Federal Republic of Germany to draw conclusions about societal structures. This project investigates the relationship between sociodemographic characteristics and income, using probabilistic and tree-based models to classify income levels. The main goal is to classify income as accurately as possible using probabilistic, decision tree, and tree-based models. In addition, decision tree and probabilistic models are employed, as they offer good interpretability and a high mental fit. This allows for the extraction of information on influencing factors that more complex models, such as neural networks or ensemble learning methods, do not easily provide.

### Hypothesis
Income is largely influenced by factors over which individuals have limited control – such as gender, nationality, or level of education. Those factors contain sufficient predictive information to classify household main income earners into different income classes with a significantly higher accuracy than random guessing or predicting the majority class. Tree-based and probabilistic models will outperform baseline methods and provide insights.

### Dataset: CAMPUS file from the 2010 Microcensus (fully anonymized and designed for academic/student use)
The CAMPUS file is a 3.5% sample of the 2010 Microcensus, containing data on 23,374 individuals from 11,494 households. In total, 427 of the original 828 features are included in the dataset. The dataset is structured into several parts, one of which contains information about the main income earner. To obtain specific information on the main income earner, only the features from this section were selected. In addition, the federal state in which the household is located was included. This results in the following 12 features.

- **The analyzed features**:
  federal state (east, west), gender, citizenship, marital status, job, employment status, employment sector, employment position, livelihood, educational qualification, highest qualification, primary residence, household relationship
Each feature except federal state is primarly connected to the main income earner of a household
- **Target variable:**  
  Income of the main income earner.






## 2. Data Preparation



- The data was loaded from the CAMPUS file of the 2010 Microcensus using the `data_load()` function.
- The script `transformation.py` performs the data transformation.

### The following steps were carried out:

1. Column names were replaced with descriptive labels.
2. A dataset with human-readable feature values was created and saved as *df_labels.csv*.
   - This version retains missing values to allow analysis of main income earners without income in the preprocessing section.
3. Normalization and removal of missing values.
4. Removal of all samples with the following income classes, as they are not comparable:
   - 50: *Self-employed farmer*
   - 90: *No income*
   - 99: *Not specified*
6. Application of `train_test_split` from the Scikit-learn library:
   - *test_size = 0.3 → Train: (10,376 × 13) – Test: (4,447 × 13)*
7. Saving of the transformed datasets and mappings in the `/data/` directory.



# 3. Data Exploration

- All results in this chapter are derived from `exploration.ipynb` and can be verified there.

Nearly all features the Dataset contains are categorical features without an inherent order. Qualification could be interpreted as a ordinal categorical feature but it is not clear in all cases, so it was treated as nominal to avoid introducing artificial ordering into the modeling process. Therefore typical explorational methods like correlation analysis are not possible. The target variable `income` is already structured as 24 different classes. A transformation into a continuous variable was avoided, as the classes are not of equal size and there is no clear upper bound. 


## 3.1 Basic Statistics:



| Feature                     | Count | Type     | Missing Values |
| -------------------------- | ----- | -------- | -------------- |
| `federal_state`            | 23,374| nominal  | 0              |
| `gender`                   | 23,107| nominal  | 267            |
| `citizenship`              | 23,107| nominal  | 267            |
| `marital_status`           | 23,107| nominal  | 267            |
| `employment_status`        | 23,107| nominal  | 267            |
| `employment_sector`        | 15,655| nominal  | 7,722          |
| `job`                      | 15,651| nominal  | 7,726          |
| `employment_position`      | 15,655| nominal  | 7,722          |
| `livelihood`               | 23,107| nominal  | 267            |
| `income`                   | 23,107| ordinal  | 267            |
| `educational_qualification`| 23,107| ordinal  | 267            |
| `highest_qualification`    | 23,079| ordinal  | 295            |
| `primary_residence`        | 23,107| metric   | 267            |
| `household_relationship`   | 23,107| nominal  | 267            |


## 3.2 Analysis of the Missing Values:

The dataset contains 267 missing values for every feature, that is primarily connected to the main income earner. originate from individuals living in collective accommodations (*Gemeinschaftsunterkünfte*), according to the source documentation.
Removing these samples is justified, as they provide no useful information for the analysis.





<img src="plots/Countplots with and without employment.png" width="1200">

The plots compare the distribution of several features for individuals with a job (top row) versus for individuals without a job (bottom row). Employend induviduals generally have higher incomes, while the income distribution for those without a job is shifted toward lower income classes. For livelihood, pensions and transfer payments make up the largest share for those without a job. In terms of marital status, most individuals in both groups are married, but the unemployed group has a higher proportion of widowed and divorced individuals. As stated in chapter 2, all rows containing only null values are removed. For individuals without employment, income often originates from pensions, unemployment benefits, or other transfers, which follow different rules and are less or in a other way dependent on characteristics like gender, education. Including them would introduce noise and potentially weaken the relationship between the predictive features and income, thereby reducing the model’s ability to identify meaningful patterns.

# 4. Income Clustering

`02_income-clustering.ipynb` discusses two methods to 1D-Clustering. Firstly, 1D K-Means is a simple method that determines cluster centers based on the Euclidean distance between points, weighted by the counts for each class. Secondly, Gaussian Mixture Model (GMM) clustering is a probabilistic approach that fits 
K Gaussian distributions to the data.

## 4.1 1D K-Means Clustering

K-Means is a unsupervised clustering algorithm that minimizes euclidean distances. The 1D-KMeans splits the variable into several groups with the same size. To better reflect reality, the number of observations in each original class is used as a weight. As a result, the cluster centers shift toward more frequent classes, and the clusters are no longer equally wide. The number of clusters must be set in advance and therefore needs to be analyzed. The silhouette score is well suited for this purpose.

<img src="plots/silhouette scores.png" width="422">
<img src="plots/k4 silhouette.png" width="400">

Here, it becomes evident that the score increases with K, with K=4 and K=7 being particularly interesting because the silhouette score shows a notable jump. The right-hand plot shows an example silhouette diagram for a K=4 cluster solution. It clearly illustrates that all clusters are well separated and that there are no misclassifications.





## 4.2 1D GMM Clustering

Gaussian Mixture Model (GMM) clustering is also an unsupervised clustering algorithm, but unlike K-Means, it does not produce hard assignments. Instead, it provides probabilistic membership scores for each cluster. In contrast to K-Means, a Gaussian Mixture Model assumes that the data is generated from a mixture of K Gaussian Distributions. AIC is a good indicator for predicting K.
AIC = Akaike Information Criterion

<img src="plots/AIC Scores.png" width="400">

The plot shows the AIC values for different numbers of clusters. For K <= 6, the AIC is relatively high and stable but decreases sharply at K=7. This indicates, that K=7 could be a suitable Number of Clusters. Furthermore GMM-Clustering provides probabilities, rather than sharp assignments.

<img src="plots/income_classes_gmm_k7.png" width="600">
<img src="plots/gmm_membership_k7.png" width="700">

The left plot shows hard the assignent (highest probability). The right plot shows the cluster probabilities for each original income class. The second plot gives more insight into the clustering decision than k-means could provide. 

To ensure a good clustering solution, the trade-off between AIC or Silhouette Score and the number of clusters is crucial. While more clusters tend to improve the Silhouette Score and AIC, the overall model quality and interpretability decrease. Therefore, domain knowledge is also applied.  

- [The poverty line in Germany is approximately €1,300](https://biaj.de/archiv-materialien/2026-eurostat-armutsgefaehrdung-vor-und-nach-sozialleistungen-in-der-bundesrepublik-deutschland-2023.html)  
- [The average income in Germany is around €2,500](https://www.bpb.de/kurz-knapp/zahlen-und-fakten/sozialbericht-2024/553205/einkommen-und-einkommensverteilung/)

The interval of the original class 7 (€1100 – €1300) is accurately captured by the GMM.
Similarly, the interval of the original class 12 (€2300 – €2600) is well represented within the 7-cluster solution of the GMM. For the subsequent analysis steps, two clustering solutions are used. A K=4 cluster solution, which is important for the interpretability of the results. In addition, a K=7 cluster solution is applied, as this one fits the data best.

*Further Plots show results for K=4*

# 5. Feature Engineering

In `03_feature_engineering.ipynb`, mutual information and the chi-square test are evaluated. Both measures are particularly important when analyzing nominal data, as they allow us to assess the degree of association between categorical variables. Mutual information captures the amount of shared information between variables, while the chi-square test evaluates whether observed distributions significantly deviate from what would be expected under independence. Together, these methods provide complementary insights: they help identify relevant relationships among nominal features and ensure that potential dependencies in the data are not overlooked.

The following plot illustrates the amount of shared information between the features and the target variable. The feature `job` emerges as the most important predictor for income classes. Due to the ISCO-88 coding of the `job` feature, it is possible to extract both coarse and fine-grained `job` features. Specifically, the first digit of the three-digit `job` code represents the coarse grouping, while the first two digits together define the finer grouping. The resulting features have significantly fewer categories, making them more suitable for interpretation and for constructing cross-features.

<img src="plots/mutual_inf.png" width="400">

## 5.1 Baseline Model

In order to evaluate the effect of feature engineering methods, a baseline model (Categorical Naive Bayes) is trained. The model achieves the following performance.

accuracy: 0.4179 ± 0.0048 | f1_macro: 0.3686 ± 0.0059 | neg_log_loss: -1.5494 ± 0.0210

## 5.2 Cross Features

To reduce dimensionality and improve interpretability, only features up to ``federal state`` were considered for the creation of cross-features, as the mutual information of subsequent features proved to be very weak. Furthermore, instead of using the original ``job`` variable, the aggregated feature ``job_major`` was applied, representing the one-digit grouping of the ISCO-88 code. This approach ensures that the resulting cross-features remain manageable in terms of the number of categories while still preserving relevant information for the analysis.

<img src="plots/mutual_inf_cr_f.png" width="400">

Each cross-feature was added individually to the dataset, after which the baseline model was retrained. Unfortunately, none of the cross-features led to an improvement compared to the original feature set. A test using a Random Forest as the baseline model also showed no improvements resulting from the feature engineering steps, indicating that the applied feature engineering techniques did not add predictive value beyond the baseline.

| Feature Pair                            | Accuracy (±)    | F1 Macro (±)    | Neg. Log Loss (±) |
| --------------------------------------- | --------------- | --------------- | ----------------- |
| Baseline  | 0.5919 ± 0.0072 | 0.5329 ± 0.0066 | -1.3038 ± 0.0202 |
| (`job_major`, `employment_sector`)      | 0.5869 ± 0.0058 | 0.5265 ± 0.0066 | -1.5182 ± 0.0287  |
| (`job_major`, `employment_position`)    | 0.5922 ± 0.0057 | 0.5373 ± 0.0090 | -1.5014 ± 0.0306  |
| (`job_major`, `highest_qualification`)  | 0.5892 ± 0.0056 | 0.5298 ± 0.0059 | -1.5123 ± 0.0352  |
| (`job_major`, `marital_status`)         | 0.5909 ± 0.0066 | 0.5284 ± 0.0086 | -1.4757 ± 0.0267  |
| (`job_major`, `federal_state`)          | 0.5865 ± 0.0064 | 0.5291 ± 0.0073 | -1.5066 ± 0.0275  |
| (`job_major`, `gender`)                 | 0.5822 ± 0.0078 | 0.5256 ± 0.0090 | -1.5002 ± 0.0298  |
| (`highest_qualification`, `livelihood`) | 0.5911 ± 0.0051 | 0.5334 ± 0.0041 | -1.4341 ± 0.0232  |



# 6. Baseline Classifier

In `04_baseline_classifier.ipynb`, two baseline models were implemented, a random guesser and a majority guesser. These baselines establish a lower bound for model performance and allow for assessing whether the probabilistic and tree-based models achieve statistically meaningful improvements beyond trivial prediction strategies.

# 7. Naive Bayes Classifier

The Naive Bayes classifier is a probabilistic model based on Bayes’ theorem, with the central assumption that all features are conditionally independent given the class labels. 
According to [scikit-learn documentation on Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) the Naive Bayes classifier estimates the probability of a class given a set of features by applying Bayes’ theorem under the assumption of conditional independence between features. In practice, this means that each feature contributes a piece of evidence for or against each class.  

Formally, for a class \(Y=c\) and features \(X_1, ..., X_n\), the posterior probability is computed as:  

$$
P(Y=c \mid X_1, \dots, X_n) \propto P(Y=c) \cdot \prod_{i=1}^n P(X_i \mid Y=c)
$$

- \(P(Y=c)\) is the **prior probability** of the class.  
- \(P(Xi | Y=c)\) is the **likelihood** of observing feature \(Xi\) given class \(c\).  

The denominator of Bayes’ theorem (the total probability of the features) is the same across all classes, so it can be ignored when comparing classes. To obtain actual probabilities, the scores are normalized across all classes.  

Intuitively, each feature *votes* for the class it is most consistent with. By multiplying these votes together and then normalizing, Naive Bayes produces class membership probabilities.

## 7.1 Categorical Naive Bayes

The Categorical Naive Bayes classifier operates on frequency counts similar to the chi-square test, and is commonly applied in tasks such as spam filtering. Since the dataset consists of nominal variables, this model is particularly well suited, as it estimates conditional probabilities directly from observed category frequencies. After hyperparameter tuning, the best-performing model on the test data was a Categorical Naive Bayes classifier, which achieved the following results:

Accuracy: 0.602

Macro-F1: 0.551

Weighted-F1: 0.604

Log-Loss: 1.008

The detailed classification report is provided below:
| Class             | Precision | Recall | F1-Score | Support |
| ----------------- | --------- | ------ | -------- | ------- |
| 0                 | 0.73      | 0.70   | 0.72     | 1741    |
| 1                 | 0.55      | 0.51   | 0.53     | 1581    |
| 2                 | 0.51      | 0.60   | 0.55     | 986     |
| 3                 | 0.39      | 0.42   | 0.41     | 139     |
| **Accuracy**      |           |        | 0.60 | 4447    |
| **Macro Avg.**    | 0.55      | 0.56   | 0.55     | 4447    |
| **Weighted Avg.** | 0.61      | 0.60   | 0.60     | 4447    |


## 7.2 Analysis of Uncertainty

<img src="plots/connection between p and entropy.png" width="500">

The plot illustrates the relationship between the maximum predicted class probability ``max_probability`` and the corresponding prediction uncertainty measured by ``entropy``. As expected, lower maximum probabilities are associated with higher entropy values, reflecting greater uncertainty in the model’s predictions. Misclassifications *blue points* are concentrated in regions where the predicted maximum probability falls below 0.5 and entropy often exceeds 1.0, indicating a lack of confidence. In contrast, correct classifications *orange points* are more frequent in areas with higher maximum probabilities and lower entropy. When restricting the evaluation to more confident predictions, defined as those with an entropy below 0.7 and a maximum probability above 0.5, the accuracy rises to ``68.27%``.

<img src="plots/box_plots entropy.png" width="500">

The model exhibits consistently lower entropy when predicting instances of class 0, which indicates a higher level of confidence in these predictions. In particular, the classifier is most certain when assigning observations to the lowest income group. This suggests that class 0 may possess distinctive feature patterns that set it apart from the other classes, making it easier for the model to separate.

<img src="plots/conditional_probs_educational_qual.png" width="500">

The heatmap shows the conditional probabilities for the feature ``educational qualification`` across different income classes. Distinct patterns can be observed: for example, category 2 (polytechnische Oberschule DDR) is strongly associated with income class 0 (90% probability), while categories 4 and 5 (Fachhochschulreife , Abitur) shift the likelihood toward income class 2. Other categories (e.g., 8 and 9) are also predominantly linked to class 0. These clear probability distributions indicate that certain educational levels provide strong signals for distinguishing between income groups. 

This type of analysis aligns particularly well with Naive Bayes because the model is inherently probabilistic and interpretable. Each feature contributes to the class prediction through conditional probabilities estimated directly from frequency counts. As a result, Naive Bayes not only provides a predictive model but also allows for straightforward inspection of which categories drive the classification toward specific classes. This transparency makes it especially suitable for datasets with nominal variables, where feature-category relationships can be directly visualized and interpreted.


# 8. Rule Based Models 


The principal limitation of the Naive Bayes classifier arises from its assumption of conditional independence between features. To examine whether such dependencies constrain predictive performance, a Decision Tree and a Random Forest classifier were trained as benchmarks. In particular, the Random Forest can be regarded as a strong reference model, providing an approximate upper bound for the achievable performance on this dataset.


## 8.1 Decision Tree

Through grid search optimization, the best-performing Decision Tree was identified with the following parameters:
criterion = gini, max_depth = 30, min_samples_leaf = 1, and min_samples_split = 2.
The model achieved the following Scores on the train data:

Accuracy of ``0.7819 ± 0.0080``, a macro-F1 score of ``0.7558 ± 0.0091``, and a log-loss of ``4.6845 ± 0.0992``.

The Decision Tree achieved the following Scores on the test data:

Accuracy of ``0.812``, a macro-F1 score of ``0.784``, and a weighted-F1 score of ``0.812``, while the log-loss was relatively high at ``3.466``. The detailed classification report indicates strong performance across most classes:

- Class 0 was predicted with the highest reliability (precision = 0.87, recall = 0.85, F1 = 0.86).

- Class 1 and Class 2 also showed balanced performance, with F1 scores of 0.79 each.

- Class 3, however, proved more challenging, with lower recall (0.65) and F1 (0.71), likely due to the comparatively small support (139 samples).

Overall, the uncalibrated Decision Tree demonstrates robust predictive performance with relatively consistent results across classes, but its poor probability estimates (as reflected by the high log-loss) underline the limitations of uncalibrated tree models for probabilistic interpretation.

## 8.2 Calibrated Decision Tree

Probability calibration adjusts the raw model outputs so they better reflect true empirical probabilities. In practice, this ensures that a predicted probability for a class corresponds to approximately the amount of predictions that occur. Calibration typically improve log-loss, even when classification accuracy stays similar. This is because well-calibrated probabilities mitigate overconfidence or underconfidence in the predictions. Calibration is also visually assessed using reliability diagrams.

After applying probability calibration, the Decision Tree model achieved an accuracy of ``0.819``, a macro-F1 score of ``0.795``, and a weighted-F1 score of ``0.819``, while the log-loss decreased to ``0.589``. Compared to the uncalibrated model, this highlights the effect of calibration: although the classification metrics such as accuracy and F1 remain largely stable, the probability estimates become much more reliable. In practical terms, the calibrated model produces probabilities that are closer to the true empirical frequencies, making them interpretable and useful for downstream decision-making, threshold optimization, and risk-sensitive applications.

<img src="plots/reliability_plot_dt.png" width="1000">

The plot compares the predicted probabilities with the observed empirical proportions across the four income classes. It clearly shows that the calibrated decision tree aligns more closely with the diagonal trend line, indicating better probability calibration. In contrast, the uncalibrated model produces systematically biased probabilities. Moreover, the results reveal that class 3 *(the highest income group)* is particularly challenging for the model, with lower calibration quality and higher deviations from the expected trend.

## 8.4 Random Forest

As the [scikit-learn documentation on Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) states, a random forest classifier is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

A standard Random Forest model and its calibrated counterpart were first optimized using a randomized grid search, followed by a more fine-grained parameter tuning with an complete grid search over a smaller parameter space. 

The uncalibrated Random Forest achieved an accuracy of ``0.822``, a macro-F1 score of ``0.794``, and a weighted-F1 score of ``0.822``, with a log-loss of ``0.676``. The classification report shows consistent performance across the major classes:

Class 0 was predicted most reliably, with precision, recall, and F1 all at 0.87.

Class 1 and Class 2 also demonstrated balanced results, with F1 scores of 0.79 each.

Class 3 **(highest income group)** again proved most challenging, showing lower recall ``(0.66)`` and F1 ``(0.72)``, reflecting the difficulty of distinguishing this smaller class (139 samples).

Overall, the Random Forest exhibits strong predictive performance with stable results across most income groups.

The calibrated Random Forest achieves even better log-loss: ``0.479``

<img src="plots/reliability_plot_rf.png" width="1000">

The plots demonstrate that the Random Forest is already relatively well-calibrated by default. The reliability diagram further illustrates that the predicted probabilities align closely with the actual proportion of correct predictions, indicating that the model’s probability estimates can be interpreted with a high degree of confidence.

# 9. Results




## 9.1 Comparison of all Models (4-Clusters)

| Model          | Accuracy | Macro-F1 | Weighted-F1 | Log-Loss |
|----------------|----------|----------|-------------|----------|
| Random_Guess   | 0.2541   | 0.2271   | 0.2811      | 1.3863   |
| Majority_Guess | 0.3915   | 0.1407   | 0.2203      | 21.9326  |
| CategoricalNB  | 0.6024   | 0.5508   | 0.6037      | 1.0076   |
| MultinomialNB  | 0.6024   | 0.5508   | 0.6037      | 1.0075   |
| ComplementNB   | 0.6013   | 0.4730   | 0.5889      | 1.0497   |
| DT_uncal       | 0.8116   | 0.7845   | 0.8116      | 3.4663   |
| DT_cal         | 0.8190   | **0.7945** | 0.8190    | 0.5892   |
| RF_uncal       | 0.8224   | 0.7941   | 0.8222      | 0.6755   |
| RF_cal         | **0.8226** | 0.7930   | **0.8224** | **0.4794** |


With 4 income classes, the models achieve their overall best performance. Calibrated Random Forest delivers the highest Accuracy ``(0.8226)`` and Weighted-F1 ``(0.8224)``, while DT_cal slightly leads in Macro-F1 ``(0.7945)``. Calibration has a strong effect on the Decision Tree: Log-Loss drops dramatically from 3.4663 to 0.5892, while Accuracy and F1 also improve slightly. For the Random Forest, calibration has a smaller impact since it is already relatively well-calibrated by default. Still, Log-Loss improves (0.6755 -> 0.4794), making the probability estimates more reliable.

## 9.2 Comparison of all Models (7-Clusters)

| Model          | Accuracy | Macro-F1 | Weighted-F1 | Log-Loss |
|----------------|----------|----------|-------------|----------|
| Random_Guess   | 0.1320   | 0.1211   | 0.1427      | 1.9459   |
| Majority_Guess | 0.2795   | 0.0624   | 0.1221      | 25.9689  |
| CategoricalNB  | 0.4108   | 0.3701   | 0.3931      | 1.5970   |
| MultinomialNB  | 0.4108   | 0.3701   | 0.3931      | 1.5967   |
| ComplementNB   | 0.4048   | 0.3256   | 0.3670      | 1.6859   |
| DT_cal         | 0.7239   | 0.7214   | 0.7227      | 0.9636   |
| DT_uncal       | 0.7146   | 0.7122   | 0.7148      | 5.2302   |
| RF_uncal       | 0.7263   | **0.7232** | 0.7258    | 1.1024   |
| RF_cal         | **0.7279** | 0.7224   | **0.7270** | **0.7695** |


# 10. Conclusion



The results clearly demonstrate that tree-based models outperform the probabilistic Naive Bayes classifiers. This performance gap can likely be attributed, at least in part, to the independence assumption inherent in Naive Bayes, which does not hold in this socio-economic dataset. Nevertheless, all models significantly outperform the baseline approaches (random and majority guessers), which confirms that the predictors contain meaningful information and that the models are indeed interpretable.

Naive Bayes and Decision Trees offer the additional advantage of providing valuable insights into the relationships between features and the target variable, making them particularly useful for interpretability despite their lower predictive performance. Random Forest, used as the benchmark model, demonstrates strong classification power, achieving over 80% accuracy in the 4-class setup and above 70% in the 7-class scenario. These results underline that socio-economic factors have a substantial influence on individual income levels.

Across all models, the most important predictor was consistently job, which aligns well with theoretical expectations. While it would have been interesting to further investigate regional differences at the federal state level, the available dataset was too heavily anonymized to allow for meaningful conclusions in this dimension.

# 11. References



- All tools and methods used in this study are documented in the corresponding Jupyter notebooks.

- The report includes insights, plots, and metrics that were generated across the different notebooks.

- The dataset *fdz_mikrozensus_cf_2010* originates from *campus-file-fdz.nrw.de*. Due to data protection regulations, it cannot be made publicly available.